In [1]:
import sys
from pathlib import Path

def is_google_colab() -> bool:
    if "google.colab" in str(get_ipython()):
        return True
    return False

def clone_repository() -> None:
    !git clone https://github.com/featurestorebook/mlfs-book.git
    %cd mlfs-book

def install_dependencies() -> None:
    !pip install --upgrade uv
    !uv pip install --all-extras --system --requirement pyproject.toml

if is_google_colab():
    clone_repository()
    install_dependencies()
    root_dir = str(Path().absolute())
    print("Google Colab environment")
else:
    root_dir = Path().absolute()
    # Strip ~/notebooks/ccfraud from PYTHON_PATH if notebook started in one of these subdirectories
    if root_dir.parts[-1:] == ('airquality',):
        root_dir = Path(*root_dir.parts[:-1])
    if root_dir.parts[-1:] == ('notebooks',):
        root_dir = Path(*root_dir.parts[:-1])
    root_dir = str(root_dir) 
    print("Local environment")

# Add the root directory to the `PYTHONPATH` to use the `recsys` Python module from the notebook.
if root_dir not in sys.path:
    sys.path.append(root_dir)
print(f"Added the following directory to the PYTHONPATH: {root_dir}")
    
# Set the environment variables from the file <root_dir>/.env
from mlfs import config
#settings = config.HopsworksSettings(_env_file=f"{root_dir}/.env")
#POTREBBE ESSERE QUESTA ULTIMA RIGA VA COMMENTATA PER RUNNARE GITHUB ACTION --> LO STESSO IN NOTEBOOK 4

Local environment
Added the following directory to the PYTHONPATH: /Users/federicomercurio/mlfs-book


<span style="font-width:bold; font-size: 3rem; color:#333;">- Part 02: Daily Feature Pipeline for Air Quality (aqicn.org) and weather (openmeteo)</span>

## 🗒️ This notebook is divided into the following sections:
1. Download and Parse Data
2. Feature Group Insertion


__This notebook should be scheduled to run daily__

In the book, we use a GitHub Action stored here:
[.github/workflows/air-quality-daily.yml](https://github.com/featurestorebook/mlfs-book/blob/main/.github/workflows/air-quality-daily.yml)

However, you are free to use any Python Orchestration tool to schedule this program to run daily.

### <span style='color:#ff5f27'> 📝 Imports

In [2]:
import datetime
import time
import requests
import pandas as pd
import hopsworks
from mlfs.airquality import util
from mlfs import config
import json
import os
import warnings
warnings.filterwarnings("ignore")

## <span style='color:#ff5f27'> 🌍 Get the Sensor URL, Country, City, Street names from Hopsworks </span>

__Update the values in the cell below.__

__These should be the same values as in notebook 1 - the feature backfill notebook__


In [3]:
project = hopsworks.login(engine="python")
fs = project.get_feature_store() 
secrets = hopsworks.get_secrets_api()

# Get API key and sensor configuration
AQICN_API_KEY = secrets.get_secret("AQICN_API_KEY").value
location_str = secrets.get_secret("SENSOR_LOCATION_JSON").value
location_data = json.loads(location_str)

# Parse sensors array
sensors = location_data['sensors']
print(f"Loaded {len(sensors)} sensors")

# Get city from first sensor (all sensors are in same city)
city = sensors[0]['city']

# Hardcoded coordinates for Milan
latitude = 45.46
longitude = 9.19

today = datetime.date.today()

2025-11-15 19:14:36,479 INFO: Initializing external client
2025-11-15 19:14:36,479 INFO: Base URL: https://c.app.hopsworks.ai:443


2025-11-15 19:14:38,305 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1272011
Loaded 4 sensors


### <span style="color:#ff5f27;"> 🔮 Get references to the Feature Groups </span>

In [4]:
# Retrieve feature groups
air_quality_fg = fs.get_feature_group(
    name='air_quality',
    version=1,
)
weather_fg = fs.get_feature_group(
    name='weather',
    version=2,
)

---

## <span style='color:#ff5f27'> 🌫 Retrieve Today's Air Quality data (PM2.5) from the AQI API</span>


In [5]:
import requests
import pandas as pd

# Get PM2.5 data for all sensors
all_aq_data = []

for sensor in sensors:
    aq_today_df = util.get_pm25(
        sensor['aqicn_url'], 
        sensor['country'], 
        sensor['city'], 
        sensor['street'], 
        today, 
        AQICN_API_KEY
    )
    all_aq_data.append(aq_today_df)

# Combine all sensor data
aq_today_df = pd.concat(all_aq_data, ignore_index=True)
aq_today_df

,pm25,country,city,street,date,url
0,151.0,italy,milan,lambrate,2025-11-15,https://api.waqi.info/feed/A156526/
1,152.0,italy,milan,Via Cristoforo Gandino,2025-11-15,https://api.waqi.info/feed/A196504/
2,364.0,italy,milan,Via Federico Chopin,2025-11-15,https://api.waqi.info/feed/A74863/
3,152.0,italy,milan,Via Dei Lavoratori,2025-11-15,https://api.waqi.info/feed/A197593/


In [6]:
aq_today_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4 entries, 0 to 3
Data columns (total 6 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   pm25     4 non-null      float32       
 1   country  4 non-null      object        
 2   city     4 non-null      object        
 3   street   4 non-null      object        
 4   date     4 non-null      datetime64[ns]
 5   url      4 non-null      object        
dtypes: datetime64[ns](1), float32(1), object(4)
memory usage: 304.0+ bytes


## <span style='color:#ff5f27'> 🌦 Get Weather Forecast data</span>

In [7]:
hourly_df = util.get_hourly_weather_forecast(city, latitude, longitude)
hourly_df = hourly_df.set_index('date')

# We will only make 1 daily prediction, so we will replace the hourly forecasts with a single daily forecast
# We only want the daily weather data, so only get weather at 12:00
daily_df = hourly_df.between_time('11:59', '12:01')
daily_df = daily_df.reset_index()
daily_df['date'] = pd.to_datetime(daily_df['date']).dt.date
daily_df['date'] = pd.to_datetime(daily_df['date'])
daily_df['city'] = city
daily_df

Coordinates 45.5°N 9.25°E
Elevation 138.0 m asl
Timezone None None
Timezone difference to GMT+0 0 s


,date,temperature_2m_mean,precipitation_sum,wind_speed_10m_max,wind_direction_10m_dominant,relative_humidity_2m_mean,city
0,2025-11-15,11.35,0.6,5.692100,71.564964,93.0,milan
1,2025-11-16,11.15,3.1,4.680000,90.000000,98.0,milan
2,2025-11-17,14.30,0.0,7.968939,251.564957,75.0,milan
3,2025-11-18,10.00,0.0,4.680000,90.000000,61.0,milan
4,2025-11-19,8.25,0.0,1.138420,161.564957,62.0,milan
5,2025-11-20,8.20,0.0,2.414953,206.564987,59.0,milan
6,2025-11-21,9.45,0.0,4.452954,284.036255,62.0,milan


In [8]:
daily_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7 entries, 0 to 6
Data columns (total 7 columns):
 #   Column                       Non-Null Count  Dtype         
---  ------                       --------------  -----         
 0   date                         7 non-null      datetime64[ns]
 1   temperature_2m_mean          7 non-null      float32       
 2   precipitation_sum            7 non-null      float32       
 3   wind_speed_10m_max           7 non-null      float32       
 4   wind_direction_10m_dominant  7 non-null      float32       
 5   relative_humidity_2m_mean    7 non-null      float32       
 6   city                         7 non-null      object        
dtypes: datetime64[ns](1), float32(5), object(1)
memory usage: 380.0+ bytes


## <span style="color:#ff5f27;">⬆️ Uploading new data to the Feature Store</span>

In [9]:
# Insert new data
air_quality_fg.insert(aq_today_df)

2025-11-15 19:14:48,601 INFO: 	1 expectation(s) included in expectation_suite.
Validation succeeded.
Validation Report saved successfully, explore a summary at https://c.app.hopsworks.ai:443/p/1272011/fs/1258612/fg/1703267


Uploading Dataframe: 100.00% |██████████| Rows 4/4 | Elapsed Time: 00:01 | Remaining Time: 00:00


(Job('air_quality_1_offline_fg_materialization', 'SPARK'),
 {
   "success": true,
   "results": [
     {
       "success": true,
       "expectation_config": {
         "expectation_type": "expect_column_min_to_be_between",
         "kwargs": {
           "column": "pm25",
           "min_value": -0.1,
           "max_value": 500.0,
           "strict_min": true
         },
         "meta": {
           "expectationId": 747538
         }
       },
       "result": {
         "observed_value": 151.0,
         "element_count": 4,
         "missing_count": null,
         "missing_percent": null
       },
       "meta": {
         "ingestionResult": "INGESTED",
         "validationTime": "2025-11-15T06:14:48.000600Z"
       },
       "exception_info": {
         "raised_exception": false,
         "exception_message": null,
         "exception_traceback": null
       }
     }
   ],
   "evaluation_parameters": {},
   "statistics": {
     "evaluated_expectations": 1,
     "successful_expecta

In [10]:
# Insert new data
weather_fg.insert(daily_df, wait=True)

2025-11-15 19:14:58,853 INFO: 	3 expectation(s) included in expectation_suite.
Validation succeeded.
Validation Report saved successfully, explore a summary at https://c.app.hopsworks.ai:443/p/1272011/fs/1258612/fg/1668880


Uploading Dataframe: 100.00% |██████████| Rows 7/7 | Elapsed Time: 00:01 | Remaining Time: 00:00


(Job('weather_2_offline_fg_materialization', 'SPARK'),
 {
   "success": true,
   "results": [
     {
       "success": true,
       "expectation_config": {
         "expectation_type": "expect_column_min_to_be_between",
         "kwargs": {
           "column": "precipitation_sum",
           "min_value": -0.1,
           "max_value": 1000.0,
           "strict_min": true
         },
         "meta": {
           "expectationId": 739519
         }
       },
       "result": {
         "observed_value": 0.0,
         "element_count": 7,
         "missing_count": null,
         "missing_percent": null
       },
       "meta": {
         "ingestionResult": "INGESTED",
         "validationTime": "2025-11-15T06:14:58.000852Z"
       },
       "exception_info": {
         "raised_exception": false,
         "exception_message": null,
         "exception_traceback": null
       }
     },
     {
       "success": true,
       "expectation_config": {
         "expectation_type": "expect_column_

## <span style="color:#ff5f27;">⏭️ **Next:** Part 03: Training Pipeline
 </span> 

In the following notebook you will read from a feature group and create training dataset within the feature store
